<a href="https://colab.research.google.com/github/Eduard25/Redes/blob/master/RF_DatasetTrain1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Iniciamos entrenando nuestra red, nostros crearemos nuestro modelo pre-entrenado

Así que llamamos a nuestra base de datos

In [ ]:
import tensorflow as tf
import pathlib
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [ ]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense,Conv2D,SeparableConv2D,Dropout,Activation,MaxPooling2D,Flatten,BatchNormalization
from keras.optimizers import RMSprop, SGD, Adam
from keras import regularizers, layers
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image


In [ ]:
%pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.1/267.1 kB 34.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 10.6 MB/s eta 0:00:00


In [ ]:
import wandb
from wandb.keras import WandbCallback, WandbMetricsLogger, WandbModelCheckpoint

In [ ]:
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
epochs = 3
batch_size = 20
optimizer = Adam

In [ ]:
wandb.init(project="RF_DataPrueba1")
wandb.config.epochs = epochs
wandb.config.batch_size = batch_size
wandb.config.optimizer = optimizer

wandb: Currently logged in as: eduard25. Use `wandb login --relogin` to force relogin


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
cd /content/gdrive/MyDrive/Neuronal Networks Class/Reconocimiento Facial/CalabE/Img/img_align_celeba/img_align_celeba

/content/gdrive/MyDrive/Neuronal Networks Class/Reconocimiento Facial/CalabE/Img/img_align_celeba/img_align_celeba


In [ ]:
ls

In [ ]:
#Acomodamos los atributos
file_path = os.path.join('/content/gdrive/MyDrive/Neuronal Networks Class/Reconocimiento Facial/CalabE/Anno', 'list_attr_celeba.txt')

with open('/content/gdrive/MyDrive/Neuronal Networks Class/Reconocimiento Facial/CalabE/Anno/list_attr_celeba.txt', 'r') as f:
    print("skipping : " + f.readline())
    print("skipping headers : " + f.readline())

    # Luego, escribe el archivo en la ruta especificada
    with open('/content/gdrive/MyDrive/Neuronal Networks Class/Reconocimiento Facial/CalabE/Anno/attr_celeba_prepared.txt', 'w') as newf:
        for line in f:
            new_line = ' '.join(line.split())
            newf.write(new_line)
            newf.write('\n')

df = pd.read_csv('/content/gdrive/MyDrive/Neuronal Networks Class/Reconocimiento Facial/CalabE/Anno/attr_celeba_prepared.txt', sep=' ', header = None)

skipping : 202599

skipping headers : 5_o_Clock_Shadow Arched_Eyebrows Attractive Bags_Under_Eyes Bald Bangs Big_Lips Big_Nose Black_Hair Blond_Hair Blurry Brown_Hair Bushy_Eyebrows Chubby Double_Chin Eyeglasses Goatee Gray_Hair Heavy_Makeup High_Cheekbones Male Mouth_Slightly_Open Mustache Narrow_Eyes No_Beard Oval_Face Pale_Skin Pointy_Nose Receding_Hairline Rosy_Cheeks Sideburns Smiling Straight_Hair Wavy_Hair Wearing_Earrings Wearing_Hat Wearing_Lipstick Wearing_Necklace Wearing_Necktie Young 



In [ ]:
print("-----------")
print(df[0].head())
print(df.iloc[:,1:].head())
print("----------")

-----------
0    000001.jpg
1    000002.jpg
2    000003.jpg
3    000004.jpg
4    000005.jpg
Name: 0, dtype: object
   1   2   3   4   5   6   7   8   9   10  ...  31  32  33  34  35  36  37  \
0  -1   1   1  -1  -1  -1  -1  -1  -1  -1  ...  -1   1   1  -1   1  -1   1   
1  -1  -1  -1   1  -1  -1  -1   1  -1  -1  ...  -1   1  -1  -1  -1  -1  -1   
2  -1  -1  -1  -1  -1  -1   1  -1  -1  -1  ...  -1  -1  -1   1  -1  -1  -1   
3  -1  -1   1  -1  -1  -1  -1  -1  -1  -1  ...  -1  -1   1  -1   1  -1   1   
4  -1   1   1  -1  -1  -1   1  -1  -1  -1  ...  -1  -1  -1  -1  -1  -1   1   

   38  39  40  
0  -1  -1   1  
1  -1  -1   1  
2  -1  -1   1  
3   1  -1   1  
4  -1  -1   1  

[5 rows x 40 columns]
----------


In [ ]:
attributes=df.iloc[:,1:].to_numpy()
attributes[attributes==-1]=0

In [ ]:
files = tf.data.Dataset.from_tensor_slices(df[0])
attributes = tf.data.Dataset.from_tensor_slices(df.iloc[:,1:].to_numpy())
data = tf.data.Dataset.zip((files, attributes))
print(data)

<_ZipDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(40,), dtype=tf.int64, name=None))>


In [ ]:
import zipfile

In [ ]:
zip_file_path = '/content/gdrive/MyDrive/Neuronal Networks Class/Reconocimiento Facial/CelebA/Img/img_align_celeba.zip'
# Directorio de destino para extraer las imágenes
extracted_dir_path = '/content/gdrive/MyDrive/Neuronal Networks Class/Reconocimiento Facial/CalabE/Img/img_align_celeba/'

# Descomprimir el archivo ZIP
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_dir_path)

In [ ]:
#Acomodamos las imágenes
path_to_images = '/content/gdrive/MyDrive/Neuronal Networks Class/Reconocimiento Facial/CalabE/Img/img_align_celeba/img_align_celeba/'
def process_file(file_name, attributes):
    image = tf.io.read_file(path_to_images + file_name)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [192, 192])
    image /= 255.0  #
    return image, attributes

labeled_images = data.map(process_file)

print(labeled_images)

<_MapDataset element_spec=(TensorSpec(shape=(192, 192, 3), dtype=tf.float32, name=None), TensorSpec(shape=(40,), dtype=tf.int64, name=None))>


In [ ]:
Total = len(labeled_images)
print(Total)

202599


In [ ]:
images = labeled_images.shuffle(buffer_size=10*batch_size)
images = images.batch(batch_size)
train_images = images.take(int(0.8*Total))
test_images = images.skip(int(0.8*Total))

Ahora creamos el modelo para entrenar la red

In [ ]:
inputs = keras.Input(shape=(192,192,3))
x = layers.Rescaling(1./255)(inputs)
x = layers.Conv2D(filters=32, kernel_size=5, use_bias=False)(x)

In [ ]:
for size in (32, 64, 128, 256, 512):
    residual = x
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    x = layers.SeparableConv2D(size, 3, padding="same", use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    x = layers.SeparableConv2D(size, 3, padding="same", use_bias=False)(x)
    x = layers.MaxPooling2D(3, strides=2, padding="same")(x)
    residual = layers.Conv2D(size, 1, strides=2, padding="same", use_bias=False)(residual)
    x = layers.add([x, residual])

In [ ]:
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(256, activation="sigmoid")(x)
outputs = layers.Dense(40, activation="sigmoid")(x)
model = keras.Model(inputs=inputs, outputs=outputs)

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 192, 192, 3)]        0         []                            
                                                                                                  
 rescaling (Rescaling)       (None, 192, 192, 3)          0         ['input_1[0][0]']             
                                                                                                  
 conv2d (Conv2D)             (None, 188, 188, 32)         2400      ['rescaling[0][0]']           
                                                                                                  
 batch_normalization (Batch  (None, 188, 188, 32)         128       ['conv2d[0][0]']              
 Normalization)                                                                               

In [ ]:
model.compile(optimizer=Adam(), loss='binary_crossentropy')
model.fit(train_images, batch_size=batch_size, epochs=epochs, validation_data=test_images, callbacks=[WandbCallback()])
model.save("DataEntrenada2.hf5")

Epoch 1/3
 1119/10130 [==>...........................] - ETA: 10:44:56 - loss: 0.3795

Este ultimo error, hay que checar la ruta sobre todo el nombre del archivo no debe de tener el img_align_celeba solamente el número